<a href="https://colab.research.google.com/github/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/blob/main/Chapter12/Detectron2_Chapter12_Export.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
import torch
print(torch.__version__)

1.13.1+cu116


In [ ]:
!python -m pip install \
'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-zml6kvfz
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-zml6kvfz
  Resolved https://github.com/facebookresearch/detectron2.git to commit 1523b3e9929a06d798871eb9afc4c9f770743baf
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Dataset

In [ ]:
!wget -q https://github.com/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/raw/main/datasets/braintumors_coco.zip
!unzip -q braintumors_coco.zip

# Get a predictor

In [ ]:
!wget https://github.com/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/raw/main/Chapter08/object_detector_hook.zip
!unzip object_detector_hook.zip

--2023-02-13 17:23:46--  https://github.com/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/raw/main/Chapter08/object_detector_hook.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/main/Chapter08/object_detector_hook.zip [following]
--2023-02-13 17:23:46--  https://media.githubusercontent.com/media/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/main/Chapter08/object_detector_hook.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 307074938 (293M) [application/zip]
Saving to: ‘object_detector_hook.

In [ ]:
output_dir = "output/object_detector_hook/"

In [ ]:
!unzip {output_dir}model_best.pth.zip

Archive:  output/object_detector_hook/model_best.pth.zip
  inflating: output/object_detector_hook/model_best.pth  


In [ ]:
import torch
import detectron2
import pickle
with open(output_dir+"cfg.pickle", "rb") as f:
  cfg = pickle.load(f)
cfg.MODEL.WEIGHTS = output_dir+"model_best.pth"
device = "cuda" if torch.cuda.is_available() else "cpu"
cfg.MODEL.DEVICE = device

In [ ]:
from detectron2.utils.logger import setup_logger
logger = setup_logger()
from detectron2.engine import DefaultPredictor
predictor = DefaultPredictor(cfg)
model = predictor.model

[02/13 17:24:07 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from output/object_detector_hook/model_best.pth ...


## Register the test dataset

In [ ]:
name_ds = "braintumors_coco"
name_ds_test = name_ds + "_test"
image_root_test = name_ds + "/test"
af = "_annotations.coco.json"
json_file_test = name_ds + "/test/" + af

from detectron2.data.datasets import register_coco_instances
register_coco_instances(
    name = name_ds_test,
    metadata = {},
    json_file = json_file_test,
    image_root = image_root_test
    )


In [ ]:
from detectron2.data import (
    build_detection_test_loader,
)
tld = build_detection_test_loader(cfg, cfg.DATASETS.TEST[0])
tli = iter(tld)

[02/13 17:24:07 d2.data.datasets.coco]: Loaded 223 images in COCO format from braintumors_coco/test/_annotations.coco.json
[02/13 17:24:07 d2.data.build]: Distribution of instances among all 2 categories:
|  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|
|  negative  | 154          |  positive  | 87           |
|            |              |            |              |
|   total    | 241          |            |              |
[02/13 17:24:07 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[02/13 17:24:07 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[02/13 17:24:07 d2.data.common]: Serializing 223 elements to byte tensors and concatenating them all ...
[02/13 17:24:07 d2.data.common]: Serialized dataset takes 0.06 MiB


In [ ]:
def get_model_inputs(tli):
  inputs = next(tli)
  inputs = [{"image": input["image"] for input in inputs}]  
  return inputs

In [ ]:
inputs = get_model_inputs(tli)
test_inputs = get_model_inputs(tli)

In [ ]:
from detectron2.export import TracingAdapter
wrapper= TracingAdapter(model, inputs=inputs)
wrapper.eval()
traced_script_module  = torch.jit.trace(
    func              = wrapper, 
    example_inputs    = wrapper.flattened_inputs)
traced_script_module.save("traced_model.pt")

/usr/local/lib/python3.8/dist-packages/detectron2/structures/image_list.py:85: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert t.shape[:-2] == tensors[0].shape[:-2], t.shape
/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/usr/local/lib/python3.8/dist-packages/detectron2/structures/boxes.py:151: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means th

# Load back

In [ ]:
loaded_model = torch.jit.load("traced_model.pt")
loaded_model(test_inputs[0]["image"])

(tensor([[456.4723, 163.2672, 580.1592, 317.8206],
         [456.2765, 154.8416, 584.7808, 311.3090],
         [492.0383, 161.2917, 578.7090, 254.3066],
         [484.3123, 161.7378, 575.2621, 264.6025],
         [451.0178, 200.9709, 546.3406, 313.3496],
         [461.3992, 195.3202, 546.2075, 306.3005]], device='cuda:0',
        grad_fn=<IndexBackward0>),
 tensor([0, 1, 1, 0, 1, 0], device='cuda:0'),
 tensor([0.7608, 0.3361, 0.1744, 0.1045, 0.0825, 0.0806], device='cuda:0',
        grad_fn=<IndexBackward0>),
 tensor([800, 800]))

In [ ]:
predictor.model(test_inputs)

[{'instances': Instances(num_instances=6, image_height=800, image_width=800, fields=[pred_boxes: Boxes(tensor([[456.4723, 163.2672, 580.1592, 317.8206],
          [456.2765, 154.8416, 584.7808, 311.3090],
          [492.0383, 161.2917, 578.7090, 254.3066],
          [484.3123, 161.7378, 575.2621, 264.6025],
          [451.0178, 200.9709, 546.3406, 313.3496],
          [461.3992, 195.3202, 546.2075, 306.3005]], device='cuda:0',
         grad_fn=<IndexBackward0>)), scores: tensor([0.7608, 0.3361, 0.1744, 0.1045, 0.0825, 0.0806], device='cuda:0',
         grad_fn=<IndexBackward0>), pred_classes: tensor([0, 1, 1, 0, 1, 0], device='cuda:0')])}]

## Convert to rich output (instances)

In [ ]:
wrapper.outputs_schema(
    loaded_model(test_inputs[0]["image"]))

[{'instances': Instances(num_instances=6, image_height=800, image_width=800, fields=[pred_boxes: Boxes(tensor([[456.4723, 163.2672, 580.1592, 317.8206],
          [456.2765, 154.8416, 584.7808, 311.3090],
          [492.0383, 161.2917, 578.7090, 254.3066],
          [484.3123, 161.7378, 575.2621, 264.6025],
          [451.0178, 200.9709, 546.3406, 313.3496],
          [461.3992, 195.3202, 546.2075, 306.3005]], device='cuda:0',
         grad_fn=<IndexBackward0>)), pred_classes: tensor([0, 1, 1, 0, 1, 0], device='cuda:0'), scores: tensor([0.7608, 0.3361, 0.1744, 0.1045, 0.0825, 0.0806], device='cuda:0',
         grad_fn=<IndexBackward0>)])}]

## Scripting

In [ ]:
from detectron2.export import scripting_with_instances
from torch import Tensor
from detectron2.structures import Boxes
fields = {
    "proposal_boxes": Boxes,
    "objectness_logits": Tensor,
    "pred_boxes": Boxes,
    "scores": Tensor,
    "pred_classes": Tensor,
    "pred_masks": Tensor,
    "pred_keypoints": Tensor,
    "pred_keypoint_heatmaps": Tensor,
}
model.eval()
scripted_model = scripting_with_instances(model, fields)

In [ ]:
ret = scripted_model.inference(
    test_inputs, 
    do_postprocess=False)[0]
print(ret)

<ipython-input-21-cf9e23b390d6>:1: UserWarning: FALLBACK path has been taken inside: runCudaFusionGroup. This is an indication that codegen Failed for some reason.
To debug try disable codegen fallback path via setting the env variable `export PYTORCH_NVFUSER_DISABLE=fallback`
 (Triggered internally at ../torch/csrc/jit/codegen/cuda/manager.cpp:331.)
  ret = scripted_model.inference(test_inputs, do_postprocess=False)[0]


In [ ]:
print(ret.pred_boxes)
print(ret.pred_classes)
print(ret.scores)

Boxes(tensor([[456.4723, 163.2672, 580.1592, 317.8206],
        [456.2765, 154.8416, 584.7808, 311.3090],
        [492.0383, 161.2917, 578.7090, 254.3066],
        [484.3123, 161.7378, 575.2621, 264.6025],
        [451.0178, 200.9709, 546.3406, 313.3496],
        [461.3992, 195.3202, 546.2075, 306.3005]], device='cuda:0',
       grad_fn=<IndexBackward0>))
tensor([0, 1, 1, 0, 1, 0], device='cuda:0')
tensor([0.7608, 0.3361, 0.1744, 0.1045, 0.0825, 0.0806], device='cuda:0',
       grad_fn=<IndexBackward0>)
